# Bleu scores on backtranslation of summary 

In [1]:
# save GEMINI_API_KEY in .env file
# call `huggingface-cli login` to login to huggingface
import os
import dotenv
import json

dotenv.load_dotenv()

False

In [33]:
source_languages = [
    "english",
    "french",
    "german",
    "hindi",
    "portuguese",
    "spanish",
]
target_languages = {
    "english": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french","swahili"],
    "french": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish",],
    "german": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french",],
    "hindi": ["bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french",],
    "portuguese": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "spanish", "french",],
    "spanish": ["hindi", "bengali", "tamil", "urdu", "arabic", "persian", "french",],
}
datasets = {}

with open(f"./codesearchnet_summary_english.json", "r") as f:
        datasets["english"] = json.load(f)
datasets["english"][0]
# datasets["hindi"][0]
print(len(datasets["english"]))

100


In [4]:
import requests
def get_ollama_completion(prompt, model="gemma3:4b-it-q4_K_M"):
    """Get completion from Ollama API"""
    try:
        response = requests.post('http://localhost:11434/api/generate',
                               json={
                                   "model": model,
                                   "prompt": prompt,
                                   "stream": False
                               })
        response.raise_for_status()  # Raise exception for bad status codes
        data = response.json()
        
        # Check if response contains the expected data
        if 'response' not in data:
            print(f"Unexpected API response format: {data}")
            return ""
            
        return data['response']
    except requests.exceptions.RequestException as e:
        print(f"Error calling Ollama API: {e}")
        return ""

In [34]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")
import bert_score

source=[entry["summary"] for entry in datasets["english"]]

target_languages = {
    "french": "fr",
    "german": "de",
    "hindi": "hi",
    "portuguese": "pt",
    "spanish": "es"
}

datasets = {}
model = "xlm-roberta-large"  # Ensure this is a string

for lang, iso_code in target_languages.items():
    with open(f"./codesearchnet_summary_{lang}.json", "r") as f:
        datasets[lang] = json.load(f)
    
    target = [entry["summary"] for entry in datasets[lang]]

    # Compute BERTScore using the correct language code
    P, R, F1 = bert_score.score(target, source, model_type=model, lang=iso_code)

    print(f"Results for {lang.capitalize()} to English:")
    print(f"Avg Precision: {P.mean():.4f}, Avg Recall: {R.mean():.4f}, Avg F1: {F1.mean():.4f}\n")
        

        

    

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Results for French to English:
Avg Precision: 0.8841, Avg Recall: 0.8838, Avg F1: 0.8839

Results for German to English:
Avg Precision: 0.8823, Avg Recall: 0.8762, Avg F1: 0.8793

Results for Hindi to English:
Avg Precision: 0.8648, Avg Recall: 0.8581, Avg F1: 0.8614

Results for Portuguese to English:
Avg Precision: 0.8903, Avg Recall: 0.8841, Avg F1: 0.8872

Results for Spanish to English:
Avg Precision: 0.8890, Avg Recall: 0.8844, Avg F1: 0.8867

